In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os
import mysql.connector
import string
import secrets
import random
import datetime



In [3]:
def clean(s):
	s.strip()
	s=re.sub("\n+","\r",s)
	s=re.sub(r"\s+"," ",s)
	return s

def digits(s):
	return re.sub("\D","",s)

def log(s):
	with open("tem.txt","w") as f:
		f.write(clean(str(s)))

def get(link):
	response=requests.get(link)
	return BeautifulSoup(response.content,"html.parser")

def generate_password(length=15):
	characters = string.ascii_letters + string.digits + string.punctuation
	password = ''.join(secrets.choice(characters) for i in range(length))
	return password

def generate_date():
	start_date = datetime.date(1970, 4, 1)
	end_date = datetime.date(2003, 4, 10)
	r=random.choice([start_date + datetime.timedelta(n) for n in range((end_date - start_date).days + 1)])
	return str(r)


In [9]:
url = "https://books.toscrape.com/"
soup=get(url)
categories={}
cats=soup.find_all("ul")[1].find_all("li")

for cat in cats:
	link=url+cat.find("a").get("href") 
	categories[cat]=link



In [10]:
articles=list(soup.find_all("article"))
nxt=soup.find("li",string="next")
link=url+nxt.find("a").get("href")
second=get(link)
articles+=second.find_all("article")
nxt=second.find("li",string="next")
while  nxt is not  None:
	link=url+"catalogue/"+second.find("a",string="next").get("href")
	second=get(link)
	articles+=second.find_all("article")
	nxt=second.find("li",string="next")



In [12]:
pages=[]

for article in articles:
	link=article.find("h3").find("a").get("href")
	if link.startswith("catalogue/"):
		link=url+link
	else:
		link=url+"catalogue/"+link
	pages.append(get(link))


In [13]:
books=[]
images=[]
for page in pages:
	book={}
	image={}
	book["title"]=clean(page.find("title").text).replace(" | Books to Scrape - Sandbox ","")
	book["category"]=clean(page.find_all("li")[2].text[1:])
	book["description"]=clean(page.find_all("p")[3].text)
	image["img"]=page.find("img").get("src")[6:]
	image["alt"]=page.find("img").get("alt")
	info=[clean(i.text) for i in page.find_all("td")]
	book["upc"]=info[0]
	image["upc"]=info[0]
	book["type"]=info[1]
	book["price"]=float(info[2][1:])
	book["tax"]=float(digits(info[4]))
	book["stock"]=int(digits(info[5]))
	book["rating"]=0
	book["reviews"]=int(digits(info[6]))
	books.append(book)
	images.append(image)



In [15]:
file=open("categories.csv","w",newline='')
writer=csv.DictWriter(file,fieldnames=["category","link"])
writer.writeheader()


for cat,link in categories.items():
	row={"category":cat,"link":link}
	writer.writerow(row)

file.close()

In [16]:
file=open("books.csv","w",newline='',encoding="utf-8")
writer=csv.DictWriter(file,fieldnames=books[0].keys())
writer.writeheader()
writer.writerows(books)
file.close()

In [20]:
with open("images.csv","w",newline='') as file:
	writer=csv.DictWriter(file,images[0].keys())
	writer.writeheader()
	writer.writerows(images)



In [ ]:
for book in books:
	path=book["img"]
	r=requests.get(url+path)
	os.makedirs(os.path.dirname(path),exist_ok=True)
	with open(path,"wb") as img:
		img.write(r.content)


In [5]:
db = mysql.connector.connect(
	host="localhost",
	user="root",
	password="ahmedfarid",
	database="store"
)

cursor = db.cursor()



In [ ]:
with open('books.csv', newline='', encoding='utf-8') as file:
	reader = csv.reader(file)
	next(reader)
	for row in reader:
		sql = "INSERT INTO product(upc, name, type, category,description, stock, price, tax,rating, reviews) VALUES ( %s , %s , %s , %s , %s , %s , %s , %s , %s ,%s)"
		values = tuple(row)
		print(int(values[5]))
		cursor.execute(sql, values)
		db.commit()
print("Data inserted successfully")


In [ ]:
with open('images.csv', newline='', encoding='utf-8') as file:
	reader = csv.reader(file)
	next(reader)
	for row in reader:
		sql = "INSERT INTO image(upc,file,alt ) VALUES ( %s , %s , %s )"
		with open(row[1],"rb") as imgFile:
			row[1]=imgFile.read()
			values = tuple(row)
			cursor.execute(sql, values)
			db.commit()
print("Data inserted successfully")


In [8]:
sql="delete from user;"

cursor.execute(sql)
db.commit()
with open("users.csv") as file:
	reader=csv.reader(file)
	next(reader)
	for row in reader:
		sql="insert into user(city,country,fname,lname,email,dob,password)values(%s,%s,%s,%s,%s,%s,%s)"
		row.append(row[2]+"."+row[3]+str(random.randrange(100000))+"@domain.com")
		row.append(generate_date())
		row.append(generate_password())
		cursor.execute(sql,tuple(row))
		db.commit()

